In [1]:
import pandas as pd

# Vamos a desarrollar un proceso de clasificación para detectar titulares de noticias falsas

## Para ello:
- Se construye un dataset de 500 instancias (250 titulares generados mediante ChatGPT 3.5 y 250 titulares tomados de Cubadebate en los 10 primeros días del mes de abril de 2024)

- Utilizaremos spacy para realizar análisis cualitativo y preprocesamiento de los titulares

- Utilizaremos scikit-learn para construir dos clasificadores y comparar sus rendimientos

## A continuación se construye el dataset a utilizar

In [2]:
df = pd.DataFrame(columns=['title_new','label'])
false_news = open('./False/news.txt',mode="r",encoding="utf-8")
#Noticias falsas
false_news_array = false_news.readlines()
for news_title in false_news_array:
    df = pd.concat([df,pd.DataFrame({"title_new":[news_title],"label":0})],axis=0,ignore_index=True)
false_news.close()
df

,title_new,label
0,"""Cuba descubre la cura definitiva para el cánc...",0
1,"""Nuevo informe revela que Cuba planea coloniza...",0
2,"""Fidel Castro resucita y lidera un golpe de Es...",0
3,"""Cuba fabrica armas nucleares en secreto"".\n",0
4,"""Descubren ciudad subterránea en Cuba habitada...",0
...,...,...
245,"""Científicos cubanos logran avances en la prod...",0
246,"""Cuba se destaca en la formación de talento hu...",0
247,"""Cubanos reciben reconocimiento internacional ...",0
248,"""Cuba implementa políticas de inclusión social...",0


In [3]:
true_news = open('./True/news.txt',mode="r",encoding="utf-8")
#Noticias reales
true_news_array = true_news.readlines()
for news_title in true_news_array:
    df = pd.concat([df,pd.DataFrame({"title_new":[news_title],"label":1})],axis=0,ignore_index=True)
true_news.close()
df

,title_new,label
0,"""Cuba descubre la cura definitiva para el cánc...",0
1,"""Nuevo informe revela que Cuba planea coloniza...",0
2,"""Fidel Castro resucita y lidera un golpe de Es...",0
3,"""Cuba fabrica armas nucleares en secreto"".\n",0
4,"""Descubren ciudad subterránea en Cuba habitada...",0
...,...,...
495,"""La Egrem de fiesta, aniversario 60""\n",1
496,"""Papa Francisco: La paz no se construye nunca ...",1
497,"""Jóvenes comunistas listos para su XII Congres...",1
498,"""Scarlett Johansson podría ser la protagonista...",1


> Revisamos el balance de las clases

In [4]:
df.label.value_counts()

label
0    250
1    250
Name: count, dtype: int64

# Importamos spacy

In [5]:
import spacy

# Módulo pequeño de spacy (por motivos de conectividad, y también porque los corpus a analizar son relativamente pequeños)

In [6]:
nlp = spacy.load("es_core_news_sm")

### Aplicamos nlp a cada título de noticia en el dataframe:

- Es importante resaltar que nlp contiene todo el pipeline de operaciones a realizar en este caso a palabras en español.

- Entre los componentes de ese pipeline se encuentran: tok2vec, morphologizer, parser, lemmatizer etc.

- Al ser un pipeline todas las operaciones de este seráan aplicadas en orden y el producto final será un vector númerico

In [9]:
df['vector'] = df['title_new'].apply(lambda x: nlp(x).vector)
df.head()

,title_new,label,vector
0,"""Cuba descubre la cura definitiva para el cánc...",0,"[-0.14059144, -0.17235631, -0.047138385, -0.60..."
1,"""Nuevo informe revela que Cuba planea coloniza...",0,"[0.5673187, -0.3511787, 0.17250234, -0.8474052..."
2,"""Fidel Castro resucita y lidera un golpe de Es...",0,"[0.39566416, -0.7797969, 0.48761624, -0.621602..."
3,"""Cuba fabrica armas nucleares en secreto"".\n",0,"[0.27940422, 0.043662667, -0.6333035, -0.66039..."
4,"""Descubren ciudad subterránea en Cuba habitada...",0,"[0.22634433, -0.3794699, 0.019433876, -0.77808..."


> Comenzaremos con la parte de Machine Learning, separaremos nuestro dataset en el conjunto de prueba y entrenamiento con 0.2 de fracción de prueba

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    df.vector,
    df.label,
    test_size=0.2,
    random_state=42
)

> Casteo que le indica a scikit-learn que el problema es de clasificación

In [16]:
y_train = y_train.astype('int')
y_test = y_test.astype('int')

> Convertimos los vectores a un formato más cómodo para el clasificador

In [13]:
import numpy as np

X_train_2d = np.stack(X_train)
X_test_2d = np.stack(X_test)

In [17]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler_train = scaler.fit_transform(X_train_2d)
scaler_test = scaler.transform(X_test_2d)

clf = MultinomialNB()
clf.fit(scaler_train,y_train)


MultinomialNB()

In [18]:
y_pred = clf.predict(scaler_test)

# Resultados del Naive Bayes

In [21]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        46
           1       1.00      0.94      0.97        54

    accuracy                           0.97       100
   macro avg       0.97      0.97      0.97       100
weighted avg       0.97      0.97      0.97       100



# Resultados KNN

In [22]:
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=5, metric='euclidean')
clf.fit(X_train_2d,y_train)
y_pred = clf.predict(X_test_2d)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95        46
           1       1.00      0.91      0.95        54

    accuracy                           0.95       100
   macro avg       0.95      0.95      0.95       100
weighted avg       0.95      0.95      0.95       100



# Conclusiones

- De forma general el performance de los algoritmos fue bastante alto, aunque el NaiveBayes resultó mejor

- No se puede obviar que la muestra es pequeña en ambas clases

- A partir de lo anterior y del origen de los datos no podemos descartar que los modelos presenten over-fitting, es decir funcionen muy bien con titulares falsos generados por ChatGPT 3.5 para el prompt utilizado, y titulares reales del sitio web Cubadebate, pero saliendo de ese dominio las métricas de los modelos deben decrementarse significativamente